Classes

In [1]:
class Field:
    def __init__(self, ftype):
        self.ftype = ftype

    def __set_name__(self, owner, name):
        self.name = name

    # note if a field is invoked from User class, e.g., User.name
    # instance argument here will be None
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance._data.get(self.name)

    def __set__(self, instance, value):
        if not isinstance(value, self.ftype):
            raise TypeError(
                f"{self.name} must be {self.ftype.__name__}"
            )
        instance._data[self.name] = value
        instance._dirty.add(self.name)

    # query operators
    def __eq__(self, value):
        return CmpExpr(self.name, "==", value)

    def __ge__(self, value):
        return CmpExpr(self.name, ">=", value)

    def __le__(self, value):
        return CmpExpr(self.name, "<=", value)
    
    ########### fill in the code to support > and < (10 points) ############

    def __lt__(self, value):
        return CmpExpr(self.name, "<", value)
    
    def __gt__(self, value):
        return CmpExpr(self.name, ">", value)

    


In [2]:
class Expr:
    def __bool__(self):
        raise TypeError("Expression cannot be used as boolean")

class CmpExpr(Expr):
    def __init__(self, field, op, value):
        self.field = field
        self.op = op
        self.value = value


In [3]:
import requests
import json
from urllib.parse import urlencode

### replace the following URL with your DB url
### note it does not end with /, that is, no slash after .com
FIREBASE_URL = "https://dsci-551-hw1-78d3e-default-rtdb.firebaseio.com" # my specific DB URL for this homework

class Document:
    endpoint = None # to be supplied by subclass

    def __init__(self, id, **kwargs):
        self.id = id # id is required
        self._data = {} # attribute-values stored locally
        self._dirty = set() # attributes to be updated on the DB server

        for k, v in kwargs.items():
            setattr(self, k, v) # self.k = v
            # note this will trigger descriptor protocol for field attribute
            # calling its __set__ method

    def save(self):
        """Persist only modified fields."""
        """That is, all fields marked as dirty."""
        """Remember to clear dirty flags after updating the data with server."""
        if not self._dirty: # Do nothing if no fields are dirty
            return

        ########### fill in the code (30 points) ############
        url = f"{FIREBASE_URL}/{self.endpoint}/{self.id}.json"
        data_to_update = {field: self._data[field] for field in self._dirty} # Only fields marked as dirty may be sent to the server
        response = requests.patch(url, json=data_to_update) # Use requests.patch to update the document
        response.raise_for_status()  # Raise an error for bad responses
        self._dirty.clear()  # Clear _dirty only after a successful update

    @classmethod
    def fetch(cls, id):
        """Fetch a document with the given id from server."""
        """Return an object of this class if found."""
        """Return None if not found."""

        ########### fill in the code (25 points) ############
        url = f"{FIREBASE_URL}/{cls.endpoint}/{id}.json"
        response = requests.get(url) # Retrieve the document from Firebase using requests.get()
        if response.status_code == 200:
            data = response.json()
            if data is None:
                return None  # Return None if the document does not exist
            return cls(id, **data)  # Return a fully initialized object of the calling class



    @classmethod
    def query(cls, expr):
        """Return a list of documents matching the query."""
        """It does not support orderBy="$key" and orderBy="$value"."""
        """It does not support limit."""

        if not isinstance(expr, CmpExpr):
            raise TypeError("Only one comparison supported")

        base_url = f"{FIREBASE_URL}/{cls.endpoint}.json"

        params = {"orderBy": json.dumps(expr.field)}

        

        if expr.op == "==":
            params["equalTo"] = json.dumps(expr.value)
        elif expr.op == ">=":
            params["startAt"] = json.dumps(expr.value)
        elif expr.op == "<=":
            params["endAt"] = json.dumps(expr.value)

        ########### fill in the code to support > and < (10 points) ############

        elif expr.op == ">":
            params["startAfter"] = json.dumps(expr.value) 
        elif expr.op == "<":
            params["endBefore"] = json.dumps(expr.value)  

        else:
            raise NotImplementedError(expr.op)

        url = f"{base_url}?{urlencode(params)}"

        ########### fill in the code (25 points) ############
        # retrieve from server and construct list of document objects

        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses
        data = response.json()
        results = []
        if data:
            for doc_id, doc_data in data.items():
                results.append(cls(doc_id, **doc_data))
        return results


class User(Document):
    endpoint = "users"

    name = Field(str)
    age = Field(int)
    gender = Field(str)

    def __repr__(self):
        return f"User(id={self.id}, data={self._data})"


Task 1 Tests - Comparison Operators in Field:

1. Make sure > works
2. Make sure < works
3. Check return object is proper CmpExpr
4. Expressions must not be usable in boolean contexts

In [4]:
# Test 1.1: Make sure > works
expr_gt = User.age > 10
assert isinstance(expr_gt, CmpExpr), "User.age > 10 should return a CmpExpr"
assert expr_gt.field == "age", "CmpExpr.field should be the field name"
assert expr_gt.op == ">", "CmpExpr.op should be '>'"
assert expr_gt.value == 10, "CmpExpr.value should be the RHS value"


# Test 1.2: Make sure < works
expr_lt = User.age < 10
assert isinstance(expr_lt, CmpExpr), "User.age < 10 should return a CmpExpr"
assert expr_lt.field == "age"
assert expr_lt.op == "<"
assert expr_lt.value == 10


# Test 1.3: Check return object is proper CmpExpr
assert type(expr_gt) is CmpExpr, "Expected exact type CmpExpr"
assert issubclass(CmpExpr, Expr), "CmpExpr should inherit from Expr"


# Test 1.4: Expressions must not be usable in boolean contexts
try:
    bool(expr_gt)
    raise AssertionError("bool(expr_gt) should have raised TypeError")
except TypeError as e:
    assert "Expression cannot be used as boolean" in str(e)

try:
    if expr_lt:  
        pass
    raise AssertionError("Using expr_lt in an if should have raised TypeError")
except TypeError as e:
    assert "Expression cannot be used as boolean" in str(e)

print("All Task 1 tests passed ✅")

All Task 1 tests passed ✅


Test 2: Task 2 - Persisting Modified Fields (save):

1. Only fields marked as dirty sent to server
2. Dirty is cleared only after successful update
3. If no fields are dirty, do nothing

In [5]:
# Helpers for mocking requests responses
class FakeRespOK:
    def raise_for_status(self):
        return None

class FakeRespFail:
    def raise_for_status(self):
        raise Exception("HTTP error")


# Test 2.1: Only fields marked as dirty sent to server
def test_save_only_sends_dirty_fields():
    original_patch = requests.patch
    called = {}

    def fake_patch(url, json=None):
        called["url"] = url
        called["json"] = json
        return FakeRespOK()

    requests.patch = fake_patch
    try:
        u = User("u1", name="Alice", age=20, gender="F")

        # Since __init__ sets fields via setattr, all are dirty at first.
        # Clear to simulate "already saved" baseline.
        u._dirty.clear()

        # Modify only one field
        u.age = 21

        u.save()

        assert called["json"] == {"age": 21}, "PATCH payload must include only dirty fields"
        assert u._dirty == set(), "Dirty should be cleared after successful save"
    finally:
        requests.patch = original_patch


# Test 2.2: Dirty cleared only after successful update
def test_dirty_not_cleared_on_failure():
    original_patch = requests.patch

    def fake_patch(url, json=None):
        return FakeRespFail()

    requests.patch = fake_patch
    try:
        u = User("u2", name="Bob", age=30, gender="M")
        u._dirty.clear()

        u.name = "Bobby"  # mark 'name' dirty
        assert "name" in u._dirty

        failed = False
        try:
            u.save()
        except Exception:
            failed = True

        assert failed, "save() should raise if update fails"
        assert "name" in u._dirty, "Dirty must NOT be cleared if update fails"
    finally:
        requests.patch = original_patch


# Test 2.3: If no fields are dirty, do nothing
def test_save_no_dirty_does_nothing():
    original_patch = requests.patch
    called = {"count": 0}

    def fake_patch(url, json=None):
        called["count"] += 1
        return FakeRespOK()

    requests.patch = fake_patch
    try:
        u = User("u3", name="Cara", age=25, gender="F")
        u._dirty.clear()  # simulate no pending changes

        u.save()

        assert called["count"] == 0, "requests.patch should not be called when _dirty is empty"
        assert u._dirty == set(), "_dirty should still be empty"
    finally:
        requests.patch = original_patch


test_save_only_sends_dirty_fields()
test_dirty_not_cleared_on_failure()
test_save_no_dirty_does_nothing()

print("All Task 2 tests passed ✅")

All Task 2 tests passed ✅


Test 3: Task 3 - Fetching a Document (fetch):

1. Return a fully initialized object
2. Return None if document does not exist

In [6]:
# Helper
class FakeGetResp:
    def __init__(self, status_code, payload):
        self.status_code = status_code
        self._payload = payload

    def json(self):
        return self._payload


# Test 3.1: Return a fully initialized object 
def test_fetch_returns_fully_initialized_object():
    original_get = requests.get

    def fake_get(url):
        # simulate existing document on server
        # Firebase would return a JSON object with fields
        payload = {"name": "Alice", "age": 22, "gender": "F"}
        return FakeGetResp(200, payload)

    requests.get = fake_get
    try:
        u = User.fetch("u1")

        # basic checks
        assert u is not None, "fetch should return an object when document exists"
        assert isinstance(u, User), "fetch should return an instance of the calling class"
        assert u.id == "u1", "returned object should have the correct id"

        # fully initialized fields (via descriptor __get__)
        assert u.name == "Alice"
        assert u.age == 22
        assert u.gender == "F"

    finally:
        requests.get = original_get


# Test 3.2: Return None if document does not exist
def test_fetch_returns_none_when_missing():
    original_get = requests.get

    def fake_get(url):
        # Firebase returns null for missing doc -> json() becomes None
        return FakeGetResp(200, None)

    requests.get = fake_get
    try:
        u = User.fetch("does_not_exist")
        assert u is None, "fetch should return None if document does not exist"
    finally:
        requests.get = original_get


test_fetch_returns_fully_initialized_object()
test_fetch_returns_none_when_missing()
print("All Task 3 tests passed ✅")

All Task 3 tests passed ✅


Test 4: Task 4 - Querying the Database (query):

1. Accept only one comparison expression
2. Translate to valid Firebase RTDB query params
3. Support for (==, >=, <=, >, <)

In [ ]:
# Helper
class FakeQueryResp:
    def __init__(self, payload):
        self._payload = payload

    def raise_for_status(self):
        return None

    def json(self):
        return self._payload


def test_query_accepts_only_one_comparison_expr():
    failed = False
    try:
        User.query("not an expr")
    except TypeError as e:
        failed = True
        assert "Only one comparison supported" in str(e)
    assert failed, "query() must raise TypeError if expr is not a CmpExpr"


def test_query_translates_params_and_supports_all_ops():
    original_get = requests.get
    captured = {}

    def fake_get(url):
        captured["url"] = url

        # return some fake server data (Firebase returns dict of id -> fields)
        payload = {
            "a1": {"name": "Alice", "age": 21, "gender": "F"},
            "b2": {"name": "Bob", "age": 22, "gender": "M"},
        }
        return FakeQueryResp(payload)

    requests.get = fake_get
    try:
        # 1) == produces equalTo
        results = User.query(User.age == 21)
        assert isinstance(results, list)
        assert len(results) == 2
        assert all(isinstance(x, User) for x in results)

        url = captured["url"]
        assert "orderBy=" in url, "query must include orderBy param"
        assert 'orderBy=%22age%22' in url, 'orderBy should be JSON-dumped field name ("age")'
        assert "equalTo=" in url, "== must translate to equalTo"
        assert "equalTo=21" in url, "equalTo should use the RHS value (JSON dumped)"

        # 2) >= produces startAt
        User.query(User.age >= 30)
        url = captured["url"]
        assert 'orderBy=%22age%22' in url
        assert "startAt=" in url, ">= must translate to startAt"
        assert "startAt=30" in url

        # 3) <= produces endAt
        User.query(User.age <= 18)
        url = captured["url"]
        assert 'orderBy=%22age%22' in url
        assert "endAt=" in url, "<= must translate to endAt"
        assert "endAt=18" in url

        # 4) > produces startAfter
        User.query(User.age > 10)
        url = captured["url"]
        assert 'orderBy=%22age%22' in url
        assert "startAfter=10" in url, "> must translate to startAfter"

        # 5) < produces endBefore
        User.query(User.age < 10)
        url = captured["url"]
        assert 'orderBy=%22age%22' in url
        assert "endBefore=10" in url, "< must translate to endBefore"

    finally:
        requests.get = original_get


test_query_accepts_only_one_comparison_expr()
test_query_translates_params_and_supports_all_ops()
print("All Task 4 tests passed ✅")

All Task 4 assert-only tests passed ✅


Given Test Cases:

In [8]:
u = User(1)
u.gender = "Male"
print(u) # note this only has gender info of User 1

# update User 1 info (kept locally) with the server
u.save()

User(id=1, data={'gender': 'Male'})


In [9]:
u = User.fetch(1) # fetch current value of User 1 from server
print(u)
# modify them locally
u.name = "Bob"
u.age = 30
u.save()  # push updates to the server
print(u)

User(id=1, data={'age': 30, 'gender': 'Male', 'name': 'Bob'})
User(id=1, data={'age': 30, 'gender': 'Male', 'name': 'Bob'})


In [10]:
u = User(2, age=20) # specify attribute-value in constructor
print(u)
u.save()
User.fetch(2)

User(id=2, data={'age': 20})


User(id=2, data={'age': 20})

In [11]:
u.age += 1 # update age locally
u.save()
User.fetch(2)

User(id=2, data={'age': 21})

In [12]:
users = User.query(User.age > 30)
users

[]

In [13]:
users = User.query(User.name <= "Mary")
users

[User(id=2, data={'age': 21}),
 User(id=1, data={'age': 30, 'gender': 'Male', 'name': 'Bob'})]